In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import numpy as np

In [ ]:
data_twitter=pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv",encoding = "latin",header = None)
data_twitter

In [ ]:
data_twitter.info()

In [ ]:
data_twitter.isnull().sum()

In [ ]:
data_twitter.describe()

In [ ]:
data_twitter.columns=["Target","id","date","flag","user","text"]

In [ ]:
data_twitter.head()

In [ ]:
data_twitter["Target"].unique()

In [ ]:
data_twitter.drop(["id","flag","user","date"],axis=1,inplace=True)

In [ ]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder

In [ ]:
sentimentals={0:"Negative",4:"Positive"}
def reverselabel(label):
    return sentimentals[label]
data_twitter["Target"]=data_twitter["Target"].apply(lambda x:reverselabel(x))

In [ ]:
data_twitter.head()


In [ ]:
sns.countplot(data_twitter["Target"])
plt.xticks(rotation=45)
plt.show()

In [ ]:
from wordcloud import WordCloud

In [ ]:
Negative=data_twitter[data_twitter["Target"]=="Negative"]

In [ ]:
negtweets=" ".join(i for i in Negative["text"])


In [ ]:
nwc = WordCloud().generate(negtweets)
plt.imshow(nwc)

In [ ]:
Positive=data_twitter[data_twitter["Target"]=="Positive"]


In [ ]:
postweets=" ".join(i for i in Positive["text"])

In [ ]:
pwc = WordCloud().generate(postweets)
plt.imshow(pwc)

In [ ]:
# Vectoriser

In [ ]:
data_twitter.head()

In [ ]:
TFIDF=TfidfVectorizer()
X =TFIDF.fit_transform(data_twitter["text"])

In [ ]:
import nltk
def mytokenizer(text): 
    text_data=text.split()
    return text_data



In [ ]:
data_twitter["Txt_tokenizer"]=data_twitter["text"].apply(lambda x:mytokenizer(x.lower()))

In [ ]:
data_twitter.head()

In [ ]:
import string
punct=list(string.punctuation)

print(punct)

In [ ]:
string.punctuation

In [ ]:
name ="  Hirdayesh     "
print(name.strip())

In [ ]:
def removePunctuation(text):
    text="".join([i for i in text if i.strip() not in string.punctuation])
    return text

In [ ]:
data_twitter["text"]=data_twitter["text"].apply(removePunctuation)

In [ ]:
data_twitter.head()

In [ ]:
stopwords=nltk.corpus.stopwords.words("english")



In [ ]:
def stopwordsRemover(text):
    text_twitter=[word for word in text if word not in stopwords]
    return text_twitter

In [ ]:
data_twitter["Txt_stopwords"]=data_twitter["Txt_tokenizer"].apply(lambda x:stopwordsRemover(x))

In [ ]:
data_twitter.head(10)

In [ ]:
ps=nltk.PorterStemmer()

In [ ]:
def myStemmer(text):
    text_twitter=[ps.stem(word) for word in text]
    return text_twitter

In [ ]:
data_twitter["Txt_Stem"]=data_twitter["Txt_stopwords"].apply(lambda x:myStemmer(x))

In [ ]:
data_twitter.head()

In [ ]:
def convertText(lst):
    str1=""
    for i in lst:
        str1+= i + " "
    return str1


In [ ]:
data_twitter["Txt_Stem"]=data_twitter["Txt_Stem"].apply(convertText)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
Predata=data_twitter["text"]+data_twitter["Txt_Stem"]

In [ ]:
Predata


In [ ]:
data_twitter_2=data_twitter.copy()

In [ ]:
data_twitter_2.head()

In [ ]:
data_twitter_2.drop(["Txt_tokenizer","Txt_stopwords"],axis=1,inplace=True)

In [ ]:
data_twitter_2.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_twitter_2["Target"]=le.fit_transform(data_twitter_2["Target"])
data_twitter_2["text"]=le.fit_transform(data_twitter_2["text"]) 
data_twitter_2["Txt_Stem"]=le.fit_transform(data_twitter_2["Txt_Stem"])

In [ ]:
vectorizer = TfidfVectorizer(stop_words = stopwords,max_features = 1000)
vectText = vectorizer.fit_transform(Predata)
print(vectorizer.get_feature_names())

In [ ]:
vfid=vectorizer.idf_

In [ ]:
vectorized_dict=dict(zip(vectorizer.get_feature_names(),vfid))

In [ ]:
vectorized_dict

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
sc = scaler.fit_transform(data_twitter_2)
TWITTER= pd.DataFrame(sc,columns = list(data_twitter_2.columns))
TWITTER

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans=[KMeans(n_clusters=i)for i in range(1,20)]
score=[kmeans[i].fit(TWITTER).score(TWITTER)for i in range(len(kmeans))]
n=range(1,20)